# TEST OF THE NON-PARALLELIZED ALGORITHM

In [1]:
import torch
import numpy as np
import cvxpy as cp
import time

def knapsack_specialized_pruning_complete(xi, v, w, C):
    b_list = []
    b = 0

    # Compute breakpoint vector x_plus
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0

        if b != C - 1:
            b_list.append(int(b))

        if b + 1 > C - 1:
            break
            
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    b_tensor = torch.tensor(b_list, dtype=torch.int32)
    x_plus[b_tensor] = 1  # Set breakpoints in x_plus vector

    # Edge case: w greater than largest v → allocate entirely to last bucket
    if(w > v[-1]):
        x = torch.zeros(C)
        x[-1] = 1

    # Edge case: w smaller than smallest v → allocate entirely to first bucket
    elif(w < v[0]):
        x = torch.zeros(C)
        x[0] = 1

    # Intermediate case: w within [v[0], v[-1]]
    else:
        # Sort indices by the ratio xi / v
        ratio = (xi / v)
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = np.concatenate([neg_sorted, pos_sorted])

        # Find valid first breakpoint with x_plus == 1 and ratio in [0,1]
        b = 0
        while(w / v[b_vector[b]] < 0 or x_plus[b_vector[b]] == 0):
            b += 1

        b_list_new = [b_vector[b]] 

        # Try to find a second valid breakpoint for convex combination
        while(w / v[b_vector[b]] > 1 or x_plus[b_vector[b]] == 0 or int(b_vector[b]) < int(b_list_new[0])):
            b += 1
            if(w / v[b_vector[b]] < 0):
                continue
            if(x_plus[b_vector[b]] == 1 and int(b_vector[b]) > int(b_list_new[0])):
                b_list_new.append(b_vector[b])
            if(len(b_list_new) > 2):
                b_list_new.pop(0)

        # If weight ratio exceeds 1, fallback to first index
        if(len(b_list_new) == 1 and w / v[b_list_new[0]] > 1):
            b_list_new[0] = 0

        x = torch.zeros(C)

        # Case: single coefficient solution
        if(len(b_list_new) == 1):
            x[b_list_new[0]] = round(float(w / v[b_list_new[0]]), 5)

        # Case: convex combination of two coefficients
        else:
            x1, x2 = torch.zeros(2, len(w), C, dtype=torch.float32)
            x1[torch.arange(len(w)), b_list_new[0]] = 1
            x2[torch.arange(len(w)), b_list_new[1]] = 1
            numerator = w - torch.matmul(x2, v)
            denominator = torch.matmul((x1 - x2), v)
            theta = numerator / denominator
            x[b_list_new[0]] = round(float(theta), 5)
            x[b_list_new[1]] = round(float(1 - theta), 5)

        # Compute objective value
        objective_values = torch.matmul(x, xi)

    # === Compute optimal multipliers ===
    if(x is None):
        lambda_opt = None  # No allocation → no multiplier
    elif(torch.all(x == 0)):
        lambda_opt = 0  # Null allocation → zero multiplier
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 2):
        # Two active components → standard finite difference formula
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True)
        diff = i[0][1] - i[0][0]
        passo = v[1] - v[0]
        lambda_opt = round(float(-1 / (diff * passo) * (xi[i[0][1]] - xi[i[0][0]])), 5)
    elif(torch.count_nonzero(torch.abs(x) > 1e-6) == 1):
        # Single active index → direct multiplier formula
        i = torch.nonzero(torch.abs(x) > 1e-6, as_tuple=True)
        lambda_opt = round(-float(xi[i[0][0]] / v[i[0][0]]), 5)
    else:
        print("ERROR: SOLUTION NOT ACCEPTABLE!")  # Sanity check

    # Final objective value
    objective_values = xi @ x

    return x, lambda_opt, objective_values, x_plus

# === MAIN SCRIPT ===
M = 1
N = 500

w0 = -0.11
r = 1.1

min_w, max_w = w0 - r, w0 + r

threshold = 1e-2

SUCCESS = 1

start_time = time.time()
for estremi_generazione_pesi in [0.2, 2]:
    a = -estremi_generazione_pesi
    b = estremi_generazione_pesi
    for C in range(5,260, 5):
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        conta_istanze_risolubili = 0
        conta_soluzioni_sotto_threshold = 0
        conta_moltiplicatori_errati = 0
        conta_soluzioni_errate = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)

            # cvxpy solution with inequality constraint (pruning)
            x_pruning = cp.Variable(C)
            equality_constraint_pruning = v @ x_pruning == w
            constraints_pruning = [
                equality_constraint_pruning,
                cp.sum(x_pruning) <= 1,
                x_pruning >= 0,
                x_pruning <= 1
            ]
            objective_pruning = cp.Minimize(xi @ x_pruning)
            prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
            prob_pruning.solve(solver=cp.ECOS)

            threshold = 0.0001
            if(x_pruning.value is not None):
                conta_istanze_risolubili += 1
                if(torch.max(x_opt) < threshold):
                    conta_soluzioni_sotto_threshold += 1
                if(not np.allclose(x_pruning.value, x_opt.numpy(), rtol=1e-5, atol=1e-2)):
                    conta_soluzioni_errate += 1
                    SUCCESS = 0
                    print(f"{i} ERRORE NELLE SOLUZIONI:")
                if(np.abs(equality_constraint_pruning.dual_value[0] - lambda_opt) > 5e-2):
                    conta_moltiplicatori_errati += 1
                    SUCCESS = 0
                    print(f"{i} ERRORE NEI MOLTIPLICATORI:")
                    print(x_pruning.value)
            #else: # da eliminare una volta verificato che i casi di soluzione non ammissibile sono gestiti correttamente
            #    print("x_opt:", x_opt)
            #    print("x_pruning.value:", x_pruning.value)
            #    print("lambda_opt:", lambda_opt)
            #    print("equality_constraint_pruning.dual_value:", equality_constraint_pruning.dual_value)
            #    print("-"*50)

        print("conta_istanze_risolubili:", conta_istanze_risolubili)
        print(f"conta_soluzioni_sotto_threshold ({threshold}):", conta_soluzioni_sotto_threshold)
        print("conta_soluzioni_errate:", conta_soluzioni_errate)
        print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)
        print("-"*60)
    print("\n"*5)

training_time = time.time() - start_time
print(f'Time taken by the test: {training_time:.2f} seconds')

if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")

C = 5, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]


/opt/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:356: FutureWarning: 
    You specified your problem should be solved by ECOS. Starting in
    CXVPY 1.6.0, ECOS will no longer be installed by default with CVXPY.
    Please either add ECOS as an explicit install dependency to your project
    or switch to our new default solver, Clarabel, by either not specifying a
    solver argument or specifying ``solver=cp.CLARABEL``. To suppress this
    warning while continuing to use ECOS, you can filter this warning using
    Python's ``warnings`` module until you are using 1.6.0.
    
  warnings.warn(ECOS_DEP_DEPRECATION_MSG, FutureWarning)


conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 10, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 1
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 15, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 20, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 1
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 25, M = 1, N = 

conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 1
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 165, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 170, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 175, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 500
conta_soluzioni_sotto_soglia (0.0001): 1
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 180, M = 1, 

conta_istanze_risolubili: 263
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 70, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 263
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 75, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 266
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 80, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 273
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 85, M = 1, N = 500, [w0-r, 

conta_istanze_risolubili: 291
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 230, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 281
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 235, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 280
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 240, M = 1, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 273
conta_soluzioni_sotto_soglia (0.0001): 0
conta_soluzioni_errate: 0
conta_moltiplicatori_errati: 0
------------------------------------------------------------
C = 245, M = 1, N = 500, [w0

# TEST OF THE PARALLELIZED ALGORITHM

In [32]:
def knapsack_specialized_pruning_parallel(xi, v, w, C):
    M = w.shape[0]

    # Step 1: Calcolo x_plus (breakpoints)
    b_list = []
    b = 0
    while True:
        delta_xi = (xi[b + 1:] - xi[b])
        delta_v = (v[b + 1:] - v[b])
        b = torch.argmin(delta_xi / delta_v) + 1 + b_list[-1] if b_list else 0
        if b != C - 1:
            b_list.append(int(b))
        if b + 1 > C - 1:
            break
    b_list.append(C - 1)
    x_plus = torch.zeros(C, dtype=torch.int32)
    x_plus[torch.tensor(b_list)] = 1

    # Preallocazioni
    x = torch.zeros(M, C)
    lambda_opt = torch.zeros(M)

    # Step 2: Classificazione dei problemi
    v0 = v[0]
    v_last = v[-1]
    mask_small = w < v0
    mask_large = w > v_last
    mask_mid = (~mask_small) & (~mask_large)

    # CASO: w > v[-1]
    x[mask_large, -1] = 1

    # CASO: w < v[0]
    x[mask_small, 0] = 1

    # CASO INTERMEDIO
    if mask_mid.any():
        M_mid = mask_mid.sum() #Numero di v[0]<w<v[-1]
        w_mid = w[mask_mid]
        ratio = xi / v
        neg_indices = torch.where(ratio < 0)[0]
        neg_sorted = neg_indices[torch.argsort(ratio[neg_indices], descending=True)]
        pos_indices = torch.where(ratio >= 0)[0]
        pos_sorted = pos_indices[torch.argsort(ratio[pos_indices])]
        b_vector = torch.cat([neg_sorted, pos_sorted], dim=0)
        ratio_b = w_mid[:, None] / v[b_vector]
        x_plus_b = x_plus[b_vector].bool()
        cond1 = (ratio_b >= 0) & x_plus_b
        valid_i0 = cond1.float() * torch.arange(C)[None, :]
        valid_i0[~cond1] = float('inf')
        i0_pos = valid_i0.argmin(dim=1)
        i0 = b_vector[i0_pos]
        v_i0 = v[i0]
        x_single = w_mid / v_i0
        invalid_i0 = x_plus[i0] == 0
        use_two = x_single > 1
        i1 = torch.full_like(i0, fill_value=-1)
        if use_two.any():
            b_vector_exp = b_vector.unsqueeze(0).expand(M_mid, -1)
            i0_exp = i0.unsqueeze(1).expand_as(b_vector_exp)
            x_plus_mask = x_plus[b_vector_exp] == 1
            greater_mask = b_vector_exp > i0_exp
            valid_mask = x_plus_mask & greater_mask
            masked_b_vector = torch.where(valid_mask, b_vector_exp, torch.full_like(b_vector_exp, C))
            i1_candidate, _ = masked_b_vector.min(dim=1)
            i1_candidate_use_two = i1_candidate[use_two]
            valid_i1_use_two = i1_candidate_use_two < C
            i0_use_two = i0[use_two]
            i1[use_two] = torch.where(valid_i1_use_two, i1_candidate_use_two, i0_use_two)

        # Costruzione x_mid
        x_mid = torch.zeros(M_mid, C)

        # Caso: uso un solo indice
        mask_one = ~use_two
        rows_one = torch.where(mask_one)[0]
        cols_one = i0[mask_one]
        x_mid[rows_one, cols_one] = torch.clamp(torch.round(w_mid[mask_one] / v[cols_one], decimals=5), 0.0, 1.0)

        # Caso: combinazione convessa
        mask_two = use_two & (i1 != i0)
        rows_two = torch.where(mask_two)[0]
        idx0 = i0[mask_two]
        idx1 = i1[mask_two]
        v0 = v[idx0]
        v1 = v[idx1]
        w_sel = w_mid[mask_two]
        theta = (w_sel - v1) / (v0 - v1)
        x_mid[rows_two, idx0] = torch.round(theta, decimals=5)
        x_mid[rows_two, idx1] = torch.round(1 - theta, decimals=5)

        x[mask_mid] = x_mid
        
    # === Calcolo vectorizzato dei moltiplicatori ===
    eps = 1e-6
    nz_mask = torch.abs(x) > eps
    nz_counts = nz_mask.sum(dim=1)
    lambda_opt = torch.zeros(x.shape[0])

    # Caso 1 valore non nullo
    m1 = torch.where(nz_counts == 1)[0]
    if m1.numel() > 0:
        submask = nz_mask[m1]
        indices = submask.nonzero(as_tuple=False) 
        i = indices[:, 1]
        lambda_opt[m1] = -xi[i] / v[i]
        lambda_opt[m1] = torch.round(lambda_opt[m1], decimals=5)

    # Caso 2 valori non nulli
    m2 = torch.where(nz_counts == 2)[0]
    if m2.numel() > 0:
        indices = nz_mask[m2].nonzero().reshape(-1, 2)
        grouped = indices.view(-1, 2, 2)
        i = grouped[:, 0, 1]
        j = grouped[:, 1, 1]
        delta_xi = xi[j] - xi[i]
        delta_idx = j - i
        passo = v[1] - v[0]
        lambda_opt[m2] = -delta_xi / (delta_idx * passo)
        lambda_opt[m2] = torch.round(lambda_opt[m2], decimals=5)

    objective_values = x @ xi
    return x, lambda_opt, objective_values, x_plus

# === MAIN SCRIPT ===
M = 100
N = 500

w0 = -0.11
r = 1.1

min_w, max_w = w0 - r, w0 + r

soglia = 1e-2

SUCCESS = 1

start_time = time.time()
for estremi_generazione_pesi in [0.2, 2]:
    a = -estremi_generazione_pesi
    b = estremi_generazione_pesi
    for C in range(5, 260, 5):
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        conta_istanze_risolubili = 0
        conta_soluzioni_sotto_soglia = 0
        conta_moltiplicatori_errati = 0
        conta_moltiplicatori_errati_con_soluzione_nulla = 0
        conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 = 0
        conta_soluzioni_errate = 0
        conta_soluzioni_errate_doppie = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi_all = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w_all = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi_all, v, w_all, C)

            for i in range(M):
                w = w_all[i].unsqueeze(0)
                x_opt2, lambda_opt2, phi_opt2, x_plus2 = knapsack_specialized_pruning_complete(xi_all, v, w, C)

                #print("x_i.value:", x_i.value)
                #print("torch.tensor(prob.value):", torch.tensor(prob.value))
                #print("phi_opt[i].to(dtype=torch.float64):", phi_opt[i].to(dtype=torch.float64))
                if x_opt2 is not None:
                    conta_istanze_risolubili += 1
                    if torch.max(x_opt[i]) < soglia:
                        conta_soluzioni_sotto_soglia += 1
                    if(not np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2)):
                        if(torch.isclose(phi_opt[i], phi_opt2)):
                            conta_soluzioni_errate_doppie += 1
                        #else:
                            #print(f"{seed},{i} ERRORE NELLE SOLUZIONI:")
                            #print("xi_all:", xi_all)
                            #print("x_i.value:", x_i.value)
                            #print("x_opt[i].numpy():", x_opt[i].numpy())                         
                        conta_soluzioni_errate += 1
                        SUCCESS = 0
                    if lambda_opt2 is not None and abs(lambda_opt2 - lambda_opt[i]) > 1e-3:
                        if(np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2) and 
                           torch.max(x_opt[i]) < soglia):
                            conta_moltiplicatori_errati_con_soluzione_nulla += 1
                        elif(np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2) and 
                             torch.isclose(torch.max(x_opt[i]), torch.tensor(1.0), atol=0.01)):
                            conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 += 1
                        #else:
                            #print(f"{seed},{i} ERRORE NEI MOLTIPLICATORI:\n\n\n")
                            #print("xi_all:", xi_all)
                            #print("x_i.value:", x_i.value)
                            #print("x_opt[i].numpy():", x_opt[i].numpy()) 
                        conta_moltiplicatori_errati += 1
                        SUCCESS = 0

        print("conta_istanze_risolubili:", conta_istanze_risolubili)
        print(f"conta_soluzioni_sotto_soglia ({soglia}):", conta_soluzioni_sotto_soglia)
        print("conta_soluzioni_errate:", conta_soluzioni_errate)
        print("conta_soluzioni_errate_doppie:", conta_soluzioni_errate_doppie)        
        print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)
        print("conta_moltiplicatori_errati_con_soluzione_nulla:", conta_moltiplicatori_errati_con_soluzione_nulla)    
        print("conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1:", conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1)            
        print("CHECK SOLUZIONI:", "OK" if conta_soluzioni_errate == conta_soluzioni_errate_doppie else "ERRORE")
        print("CHECK MOLTIPLICATORI:", "OK" if conta_moltiplicatori_errati == conta_moltiplicatori_errati_con_soluzione_nulla + conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 else "ERRORE")        
        print("-"*60)
    print("\n"*5)

training_time = time.time() - start_time
print(f'Time taken by the test: {training_time:.2f} seconds')

if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")

C = 5, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2195
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 10, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2510
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 15, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]


conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2786
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 100, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2778
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 105, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2806


conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2714
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 190, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2694
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 195, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-0.2, 0.2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 2685


conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 282
conta_soluzioni_errate: 12
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 12
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: ERRORE
CHECK MOLTIPLICATORI: ERRORE
------------------------------------------------------------
C = 25, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 295
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 30, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 300
con

conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 310
conta_soluzioni_errate: 3
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 3
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: ERRORE
CHECK MOLTIPLICATORI: ERRORE
------------------------------------------------------------
C = 115, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 313
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 120, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 300
con

conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 255
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 205, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 240
conta_soluzioni_errate: 0
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 0
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: OK
CHECK MOLTIPLICATORI: OK
------------------------------------------------------------
C = 210, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]
conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 238
conta_soluz

In [ ]:
#133 errori su 5100000

# [DEBUG] Study of instance subgroups to understand what is going wrong

In [35]:
# === MAIN SCRIPT ===
M = 100
N = 500

w0 = -0.11
r = 1.1

min_w, max_w = w0 - r, w0 + r

soglia = 1e-2

SUCCESS = 1

start_time = time.time()
for estremi_generazione_pesi in [2]:
    a = -estremi_generazione_pesi
    b = estremi_generazione_pesi
    for C in [20]:
        print(f"C = {C}, M = {M}, N = {N}, [w0-r, w0+r] = [{w0-r:.2f}, {w0+r:.2f}], [a, b] = [{a}, {b}]")    
        conta_istanze_risolubili = 0
        conta_soluzioni_sotto_soglia = 0
        conta_moltiplicatori_errati = 0
        conta_moltiplicatori_errati_con_soluzione_nulla = 0
        conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 = 0
        conta_soluzioni_errate = 0
        conta_soluzioni_errate_doppie = 0
        for seed in range(N):
            np.random.seed(seed)
            torch.manual_seed(seed)

            xi_all = torch.sort(torch.rand(C))[0]
            v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
            w_all = torch.rand(M) * (b - a) + a

            x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi_all, v, w_all, C)

            for i in range(M):
                w = w_all[i].unsqueeze(0)
                x_opt2, lambda_opt2, phi_opt2, x_plus2 = knapsack_specialized_pruning_complete(xi_all, v, w, C)

                #print("x_i.value:", x_i.value)
                #print("torch.tensor(prob.value):", torch.tensor(prob.value))
                #print("phi_opt[i].to(dtype=torch.float64):", phi_opt[i].to(dtype=torch.float64))
                if x_opt2 is not None:
                    conta_istanze_risolubili += 1
                    if torch.max(x_opt[i]) < soglia:
                        conta_soluzioni_sotto_soglia += 1
                    if(not np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2)):
                        if(torch.isclose(phi_opt[i], phi_opt2)):
                            conta_soluzioni_errate_doppie += 1
                        #else:
                            #print(f"{seed},{i} ERRORE NELLE SOLUZIONI:")
                            #print("xi_all:", xi_all)
                            #print("x_i.value:", x_i.value)
                            #print("x_opt[i].numpy():", x_opt[i].numpy())                         
                        conta_soluzioni_errate += 1
                        SUCCESS = 0
                    if lambda_opt2 is not None and abs(lambda_opt2 - lambda_opt[i]) > 1e-3:
                        print(f"\n\n\n{seed},{i} ERRORE NEI MOLTIPLICATORI:")
                        print("xi_all:", xi_all)
                        print("v:", v)
                        print("w:", w)
                        print("x_opt[i]:", x_opt[i])
                        print("x_opt2:", x_opt2)
                        print("lambda_opt[i]:", lambda_opt[i])
                        print("lambda_opt2:", lambda_opt2)                        
                        if(np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2) and 
                           torch.max(x_opt[i]) < soglia):
                            conta_moltiplicatori_errati_con_soluzione_nulla += 1
                        elif(np.allclose(x_opt2, x_opt[i], rtol=1e-5, atol=1e-2) and 
                             torch.isclose(torch.max(x_opt[i]), torch.tensor(1.0), atol=0.01)):
                            conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 += 1
                        #else:
                            #print(f"{seed},{i} ERRORE NEI MOLTIPLICATORI:\n\n\n")
                            #print("xi_all:", xi_all)
                            #print("x_i.value:", x_i.value)
                            #print("x_opt[i].numpy():", x_opt[i].numpy()) 
                        conta_moltiplicatori_errati += 1
                        SUCCESS = 0

        print("\n\n\nconta_istanze_risolubili:", conta_istanze_risolubili)
        print(f"conta_soluzioni_sotto_soglia ({soglia}):", conta_soluzioni_sotto_soglia)
        print("conta_soluzioni_errate:", conta_soluzioni_errate)
        print("conta_soluzioni_errate_doppie:", conta_soluzioni_errate_doppie)        
        print("conta_moltiplicatori_errati:", conta_moltiplicatori_errati)
        print("conta_moltiplicatori_errati_con_soluzione_nulla:", conta_moltiplicatori_errati_con_soluzione_nulla)    
        print("conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1:", conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1)            
        print("CHECK SOLUZIONI:", "OK" if conta_soluzioni_errate == conta_soluzioni_errate_doppie else "ERRORE")
        print("CHECK MOLTIPLICATORI:", "OK" if conta_moltiplicatori_errati == conta_moltiplicatori_errati_con_soluzione_nulla + conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1 else "ERRORE")        
        print("-"*60)
    print("\n"*3)

training_time = time.time() - start_time
print(f'Time taken by the test: {training_time:.2f} seconds')

if(SUCCESS == 1):
    print("\nTEST CONCLUSO CON SUCCESSO!")

C = 20, M = 100, N = 500, [w0-r, w0+r] = [-1.21, 0.99], [a, b] = [-2, 2]



226,3 ERRORE NEI MOLTIPLICATORI:
xi_all: tensor([0.0591, 0.0626, 0.0774, 0.1073, 0.1091, 0.1205, 0.2461, 0.2757, 0.2964,
        0.3749, 0.3846, 0.3888, 0.3892, 0.4415, 0.4861, 0.4864, 0.6760, 0.7381,
        0.9579, 0.9993])
v: tensor([-1.2100e+00, -1.1000e+00, -9.9000e-01, -8.8000e-01, -7.7000e-01,
        -6.6000e-01, -5.5000e-01, -4.4000e-01, -3.3000e-01, -2.2000e-01,
        -1.1000e-01, -5.9605e-08,  1.1000e-01,  2.2000e-01,  3.3000e-01,
         4.4000e-01,  5.5000e-01,  6.6000e-01,  7.7000e-01,  8.8000e-01])
w: tensor([0.8049])
x_opt[i]: tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -0.6587,
         0.0000,  1.6587,  0.0000,  0.0000])
x_opt2: tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.3413,




conta_istanze_risolubili: 50000
conta_soluzioni_sotto_soglia (0.01): 282
conta_soluzioni_errate: 12
conta_soluzioni_errate_doppie: 0
conta_moltiplicatori_errati: 12
conta_moltiplicatori_errati_con_soluzione_nulla: 0
conta_moltiplicatori_errati_con_1_componente_attiva_uguale_1: 0
CHECK SOLUZIONI: ERRORE
CHECK MOLTIPLICATORI: ERRORE
------------------------------------------------------------




Time taken by the test: 9.46 seconds


# TIME COMPARISON BETWEEN THE NON-PARALLELIZED ALGORITHM AND CVXPY​

In [ ]:
C = 24
M = 1

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(1000):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_complete(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for i in range(1000):
    # cvxpy solution with inequality constraint (pruning)
    x_pruning = cp.Variable(C)
    equality_constraint_pruning = v @ x_pruning == w
    constraints_pruning = [
        equality_constraint_pruning,
        cp.sum(x_pruning) <= 1,
        x_pruning >= 0,
        x_pruning <= 1
    ]
    objective_pruning = cp.Minimize(xi @ x_pruning)
    prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
    prob_pruning.solve(solver=cp.ECOS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")

# TIME COMPARISON BETWEEN THE PARALLELIZED ALGORITHM AND CVXPY

In [ ]:
C = 32
M = 44000
N = 10

w0 = -0.01
r = 1.32
a = -2
b = 2.2
min_w, max_w = w0 - r, w0 + r

np.random.seed(41)  
torch.manual_seed(41)

print("Start Specialized Algorithm")
start_time = time.time()
for i in range(N):
    xi = torch.sort(torch.rand(C))[0]
    v = torch.linspace(min_w, max_w - (max_w - min_w)/C, steps=C)
    w = torch.rand(M) * (b - a) + a
    x_opt, lambda_opt, phi_opt, x_plus = knapsack_specialized_pruning_parallel(xi, v, w, C)
training_time = time.time() - start_time
print(f'Time taken by the specialized algorithm: {training_time:.2f} seconds')

print("Start cvxpy")
start_time = time.time()
for j in range(N):
    for i in range(M):
        # cvxpy solution with inequality constraint (pruning)
        x_pruning = cp.Variable(C)
        equality_constraint_pruning = v @ x_pruning == w[i]
        constraints_pruning = [
            equality_constraint_pruning,
            cp.sum(x_pruning) <= 1,
            x_pruning >= 0,
            x_pruning <= 1
        ]
        objective_pruning = cp.Minimize(xi @ x_pruning)
        prob_pruning = cp.Problem(objective_pruning, constraints_pruning)
        prob_pruning.solve(solver=cp.SCS)
training_time2 = time.time() - start_time
print(f'Time taken by cvxpy: {training_time2:.2f} seconds')

print(f"\nThe Specialized Algorithm is {training_time2/training_time:.2f}X more perfomant than cvxpy")